In [2]:
import itertools as it
from itertools import chain
from sage.combinat.shuffle import ShuffleProduct


s = SymmetricFunctions(QQ).schur()  #Schur basis for symmetric functions
p = SymmetricFunctions(QQ).p()      #Power-sum symmetric functions

### Converting formal symbols (cells and permutations) to actual vectors and matrices:

def sign(s):                   #return the sign of the parmutation (-1)^s
    return Permutation(s).sign()

def rho(s):    #return the matrix associated with a permutation of the configuration points: sign(s)*rho(s)^(-1)
    if len(s)<n:
        s.append(n)
    s = Permutation(s) #convert to permutation
    return s.sign()*rep.representation_matrix(s.inverse())


def vector_form(pair): #convert a formal chain into an actual vector
    [cell,permutation] = pair
    #model a direct sum of N copies of a space of matrices by stacking them one on top of the other
    if cell[-1] == n:
        rows = topcells
    else: #cell[-1] == n-1
        rows = botcells

    mat = matrix(d*len(rows),d)      #matrix of zeros
    offset = rows.index(cell) #linearly order the cells, find position of current cell
    mat[d*offset: d*(offset+1)] = rho(permutation)    #insert "permutation" in the right place
    return mat

### Operations on cells: drop index (for differential), flips, swaps, left/right transvections, and the full cycle

def drop(k, cell):                    #dropping the k-th entry results in moving it to the last element
    new_cell = cell.copy()
    for j in range(1,len(cell)):
        if cell[j] >= k :
            new_cell[j] -= 1
    new_ordering = list(chain(range(1,k),range(k+1,cell[-1]+1), [k]))
    vect = vector_form( [new_cell, new_ordering] )
    return [vect, sign(new_ordering)*vect]  #the version without the sign is the one correcponding to partition p, and with the sign is the conjugate permutation p'
    return [vect, sign(new_ordering) * vect]  #1st entry is the vector associated with partition p, the 2nd is for the conjugate p'

def swap(j,k,cell):    #realise the automorphism that swaps two degenral cells j and k.
    if j>k:
        [j,k] = [k,j]
    new_ordering = list(chain(range(1,cell[j-1]+1) , range(cell[k-1]+1,cell[k]+1), range(cell[j]+1,cell[k-1]+1), range(cell[j-1]+1,cell[j]+1) , range(cell[k]+1,cell[-1]+1)))
    new_cell = cell.copy()
    for i in range(j,k):
        new_cell[i] = cell[i]+cell[k]-cell[k-1]-cell[j]+cell[j-1]
    vect = vector_form( [new_cell, new_ordering] )
    return [vect, sign(new_ordering) * vect]  #1st entry is the vector associated with partition p, the 2nd is for the conjugate p'

def cycle(cell):  #realize the automorphism that cycles all edges i_j -> i_{j+1}, the process proceeds by removing the last divider and adding a new first one
    last = cell[-1] - cell[-2]  # number of points on the last arc, to be added to all other positions
    new_cell = list( vector(cell) + vector( [last]*len(cell) )  )   #adding the last arc to all arcs
    new_cell.pop()  #remove the last element, which is now (n + last), the new last element will now be n.
    new_cell = [0] + new_cell  #reintroduce the leading 0.
    new_ordering = list(chain(range(cell[-2]+1,cell[-1]+1), range(1, cell[-2]+1)))
    vect = vector_form( [new_cell, new_ordering] )
    return [vect, sign(new_ordering) * vect]  #1st entry is the vector associated with partition p, the 2nd is for the conjugate p'

def flip(j, cell):         #realize the automorphism that reverses the j-th edge. Here j can be 1,...,g
    # the new ordering is 1, ... i_{j-1} | i_j i_j-1 ... i_{j-1}+1 | i_j+1 ...
    new_ordering = list(chain(range(1,cell[j-1]+1) , range(cell[j],cell[j-1],-1) , range(cell[j]+1,cell[-1]+1) ))
    # the name of the cell remains the same, but orientation gets reversed depending on number of points on the arc
    vect = (-1)^(cell[j]-cell[j-1]) * vector_form( [cell, new_ordering] )
    return [vect, sign(new_ordering) * vect]  #1st entry is the vector associated with partition p, the 2nd is for the conjugate p'

def transvection_right(j,k,cell):    #realize the transvection automorphism: e_j -> e_j*e_k
    #so the points on (e_j) get distributed between (e_j) then (e_k)
    #split (e_j) into two batches of points. The 1st batch stays on (e_j) in order,
    #2nd batch gets shuffled in with the existing points on (e_k).

    #The sum of the resulting transformations gets added together into a single vector form.

    action = [matrix(d* (len(topcells) if cell[-1]==n else len(botcells)) ,d)]*2  #initialize zero vector form (1st entry corresponds to p, 2nd to conjugate p')
    for l in range(cell[j-1],cell[j]+1): #ways to split the j-th edge (...|i_{j-1} ... l || l+1 ... i_j | ...)
        new_cell = cell.copy()
        ### Set up new dividers
        if j<k:
            # new cell has all dividers between j and k lowered by (i_j-l) steps; other dividers unchanged.
            for i in range(j,k):
                new_cell[i] -= cell[j]-l
            # shuffle the latter points from (e_j) into existing points on (e_k)
            for shuffle in ShuffleProduct(range(l+1,cell[j]+1),range(cell[k-1]+1,cell[k]+1)):
                new_ordering = list(chain( range(1,l+1), range(cell[j]+1,cell[k-1]+1) , shuffle, range(cell[k]+1,cell[-1]+1) ))
                #add resulting shuffled cell to the vector form
                vect = vector_form( [ new_cell, new_ordering ] )
                action[0] += vect
                action[1] += sign(new_ordering) * vect

        else:
            # when k<j need to add (i_j-l) to all dividers between k and j
            for i in range(k,j):
                new_cell[i] += cell[j]-l
            # shuffle latter point from (e_j) into existing ones on (e_k)
            for shuffle in ShuffleProduct(range(l+1,cell[j]+1),range(cell[k-1]+1,cell[k]+1)):
                new_ordering = list(chain( range(1,cell[k-1]+1), shuffle, range(cell[k]+1, l+1), range(cell[j]+1,cell[-1]+1) ))
                #add resulting shuffled cell to the vector form
                vect = vector_form( [ new_cell, new_ordering ] )
                action[0] += vect
                action[1] += sign(new_ordering) * vect

    return action


def transvection_left(j,k,cell): # similar to the right transvection above, but now e_j -> e_k*e_j.
    # The only difference is that now the 1st batch of points from (e_j) are shuffled into (e_k) instead of the 2nd. Otherwise, the same ideas apply.

    action = [matrix(d* (len(topcells) if cell[-1]==n else len(botcells)) ,d)]*2
    for l in range(cell[j-1],cell[j]+1):
        new_cell = cell.copy()     # the resulting cell has the j-th divider increase by l, and the k-th divider lowered by l.
        if j<k:
            for i in range(j,k):
                new_cell[i] -= l-cell[j-1]
            for shuffle in ShuffleProduct(range(cell[j-1]+1,l+1),range(cell[k-1]+1,cell[k]+1)):
                new_ordering =  list(chain( range(1,cell[j-1]+1), range(l+1,cell[k-1]+1) , shuffle, range(cell[k]+1,cell[-1]+1) ))
                vect = vector_form( [ new_cell, new_ordering ] )
                action[0] += vect
                action[1] += sign(new_ordering) * vect

        else: # when k<j
            for i in range(k,j):
                new_cell[i] += l-cell[j-1]
            for shuffle in ShuffleProduct(range(cell[j-1]+1,l+1),range(cell[k-1]+1,cell[k]+1)):
                new_ordering = list(chain( range(1,cell[k-1]+1), shuffle, range(cell[k]+1, cell[j-1]+1), range(l+1,cell[-1]+1) ))
                vect = vector_form( [ new_cell, new_ordering ] )
                action[0] += vect
                action[1] += sign(new_ordering) * vect

    return action

def diffl(cell): # the differential applied to a cell, given in vector form
    bdry = [matrix(d*len(botcells),d)] * 2 # initialize the zero vector form.
    for i in range(1,g+1):  #consider the i-th arc
        if cell[i] - cell[i-1] > 1:        #arcs with one point or less do not contribute to the boundary
            drop_first = drop(cell[i-1]+1,cell)
            drop_last = drop(cell[i],cell)
            bdry[0] += drop_last[0] - drop_first[0] # differential for partition p
            bdry[1] += drop_last[1] - drop_first[1] # differential for conjugate partition p'
    return bdry

def aut_matrix(word, cells): # construct a chain operator corresponding to a word in Out(F_g)
    #Input: a list of generators with leading scalar factor.
    # lT = left transvection (takes arguments j,k such that e_j -> e_k*e_j)
    # rT = right transvection (takes arguments j,k such that e_j -> e_j*e_k)
    # F = flip (takes argument j)
    # S = swap (takes arguments j,k for transposition e_j <--> e_k)
    # C = cycle (no arguments, rotates the arcs e_j -> e_{j+1})
    num_rows = d*len(cells)
    isom_mat = [word[0]*matrix.identity(num_rows)]*2
    for w in word[1:]:   # feed a list of lists, eg. [+1,["lT",1,2],["rT",1,3],["F",2],["F",3]] for complete graph K3
        w_action = [matrix(num_rows,0)]*2
        for cell in cells:
            if w[0] == "F":
                new_rel = flip(w[1],cell)
            elif w[0] == "C":
                new_rel = cycle(cell)
            elif w[0] == "S":
                new_rel = swap(w[1],w[2],cell)
            elif w[0] == "rT":
                new_rel = transvection_right(w[1],w[2],cell)
            elif w[0] == "lT":
                new_rel = transvection_left(w[1],w[2],cell)

            w_action[0] = w_action[0].augment(new_rel[0])  #calculation for partition p
            if not self_conjugate:
                w_action[1] = w_action[1].augment(new_rel[1])  #calculation for conjugate partition p'

        isom_mat[0] = isom_mat[0] * w_action[0] #calculation for partition p
        if not self_conjugate:
            isom_mat[1] = isom_mat[1] * w_action[1] #calculation for conjugate partition p'
    return isom_mat

def conf_homology(num_points, genus, isoms, codim = 1, partitions = [], Show_Progress = False):
    #Returns the character of isometry invariants of homology in codimension=*codim*, restricted to optional set of partitions
    #(default is all partitions and codimension 1)
    #Option to print out partial calculations as Schur functions - use to know which partitions are crashing.

    global n #number of points
    global g #genus
    global topcells
    global botcells
    global rep #current representation
    global d #dimension of current representation
    global self_conjugate

    n = num_points
    g = genus

    #a cell is indexed by 12...i_1 | i_1+1... i_2 | ... | i_{g-1}+1...n
    topcells = [list(chain([0],l,[n])) for l in it.combinations_with_replacement(range(0,n+1),g-1)]
    #a cell is indexed by 12...i_1 | i_1+1... i_2 | ... | i_{g-1}+1...n-1
    botcells = [list(chain([0],l,[n-1])) for l in it.combinations_with_replacement(range(0,n),g-1)]

    character = 0 # Frobenius characteristic, computed character

    if len(partitions) == 0:
        partitions = Partitions(n).list()
    else:
        partitions = [Partition(p) for p in partitions]

    for p in partitions:                 #run over the partitions~irreducibles

        pconj = p.conjugate()
        if pconj.larger_lex(p): #computing by pairs of partitions (p,p'), where p is lex-larger than its conjugate. Otherwise, skip p.
            continue
        elif p == pconj:  #self conjugate partition
            self_conjugate = True
        else:  #p is lex-larger than p', so proceed with calculation
            self_conjugate = False

        rep = SymmetricGroupRepresentation(p)   #make the Specht module associated with the partition - this assigns a matrix to every permutation
        d = rho(range(1,n+1)).ncols()           #dimension of the representation

        rel = [matrix(d*len(botcells),0)]*2     #start constructing relations matrix.
        #It starts with the differential, then continues with Id-isoms, so that kernel is top homology invariants (and coker is bottom homology coinvariants)

        # compute differential: start with matrix with 0 columns and later augment with all vector forms
        for cell in topcells:
            column = diffl(cell)
            rel[0] = rel[0].augment(column[0])
            if not self_conjugate:
                rel[1] = rel[1].augment(column[1])

        # for every isometry, add Id-isom to relations matrix
        for word in isoms:     # feed a list of lists, eg. [[+-1 (sign),["lW",1,2],["rW",1,3],["R",2],["R",3]], [["C"]]] for complete graph K3
            if codim == 1:     # augment current relations matrix by Id-isom
                isom = aut_matrix(word,botcells)
                rel[0] = rel[0].augment(matrix.identity(d*len(botcells)) - isom[0])
                if not self_conjugate:
                    rel[1] = rel[1].augment(matrix.identity(d*len(botcells)) - isom[1])

            else: #codim == 0    stack vertically the current relations matrix by Id-isom
                isom = aut_matrix(word,topcells)
                rel[0] = rel[0].stack(matrix.identity(d*len(topcells)) - isom[0])
                if not self_conjugate:
                    rel[1] = rel[1].stack(matrix.identity(d*len(topcells)) - isom[1])

        # Now relations matrix is ready. Only need to calculate its rank to determine dimension of ker and coker.
        pchar = 0
        if codim == 1:
            pchar += (d*len(botcells) - rel[0].rank())*s(p) # add the resulting multiplicity to character
            if not self_conjugate:
                pchar += (d*len(botcells) - rel[1].rank())*s(pconj)
        else: #codim 0
            pchar += (d*len(topcells) - rel[0].rank())*s(p)
            if not self_conjugate:
                pchar += (d*len(topcells) - rel[1].rank())*s(pconj)

        if Show_Progress:
            print( pchar )
        character += pchar

    return character

#### isometries to feed for graphs of genus 2 and 3
# theta graph (genus 2):
theta_isoms = [[-1,["F",1],["lT",1,2]],[-1,["C"]],[+1,["F",1],["F",2]]]

# K_4 (genus 3):
k4_isoms = [[+1,["lT",1,2],["rT",1,3],["F",2],["F",3]],[+1,["C"]]]

# can graph (genus 3):
can_isoms = [[-1,["F",1],["F",2],["F",3]],[-1,["S",1,2]],[+1,["F",2],["S",1,3],["lT",1,2],["lT",3,2]]]

# banana graph (genus 3):
banana_isoms = [[+1,["F",1],["F",2],["F",3]],[+1,["C"]],[-1,["F",3],["lT",1,3],["lT",2,3]]]

# goggles graph (genus 3):
goggles_isoms = [[+1,["S",1,3],["F",1],["F",2],["F",3]],[-1,["F",1],["lT",2,1]]]

In [7]:
print("Isometry invariants for Theta graph:")
print("n=8 points, codimension 0, all partitions")
print("Character in power-sum symmetric function:", p(conf_homology(8 ,2, theta_isoms, codim = 0, Show_Progress = True)))
print("n=10 points, codimension 1, only 4 most extremal partitions: ", conf_homology(10 ,2, theta_isoms, partitions = [[10],[9,1],[8,2],[8,1,1]]))

Isometry invariants for Theta graph:
n=8 points, codimension 0, all partitions
s[1, 1, 1, 1, 1, 1, 1, 1]
2*s[2, 1, 1, 1, 1, 1, 1]


3*s[2, 2, 1, 1, 1, 1]


5*s[6, 1, 1]


5*s[2, 2, 2, 1, 1] + 2*s[5, 3]


7*s[3, 2, 1, 1, 1] + 7*s[5, 2, 1]


...

6*s[3, 2, 2, 1] + 10*s[4, 3, 1]


9*s[3, 3, 1, 1] + 2*s[4, 2, 2]


10*s[4, 2, 1, 1]


5*s[3, 3, 2]
Character in power-sum symmetric function: 2213/20160*p[1, 1, 1, 1, 1, 1, 1, 1] + 1/90*p[2, 1, 1, 1, 1, 1, 1] - 1/96*p[2, 2, 1, 1, 1, 1] - 7/8*p[2, 2, 2, 1, 1] - 5/64*p[2, 2, 2, 2] + 1/360*p[3, 1, 1, 1, 1, 1] + 1/36*p[3, 2, 1, 1, 1] + 1/24*p[3, 2, 2, 1] + 1/9*p[3, 3, 1, 1] + 1/9*p[3, 3, 2] - 1/48*p[4, 1, 1, 1, 1] + 1/16*p[4, 2, 2] + 1/12*p[4, 3, 1] - 1/16*p[4, 4] + 1/30*p[5, 1, 1, 1] + 1/10*p[5, 2, 1] + 1/15*p[5, 3] + 2/7*p[7, 1]


n=10 points, codimension 1, only 4 most extremal partitions:  2*s[3, 1, 1, 1, 1, 1, 1, 1] + 2*s[8, 1, 1]
